# About This Notebook

This notebook was done on Kaggle and downloaded after. It contains code that we use to scrape the games data from the 40k to 50k rows of our list of games. 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/steam60kgamesdata/gamesdata.csv


In [2]:
df = pd.read_csv('/kaggle/input/steam60kgamesdata/gamesdata.csv')

In [3]:
from bs4 import BeautifulSoup
import time
import requests
import json
from tqdm import tqdm

In [4]:
counter = 0
data = []

for counter in tqdm(range(40000,50000)):
        hlink = df['Link'][counter]
        response = requests.get(hlink)
        soup1 = BeautifulSoup(response.text, 'html.parser')

        description = soup1.find('div',id='game_area_description')
        description = description.text.strip().replace('\r','').replace('\t','').replace('\n','') if description else None

        reviews = soup1.find('div', class_='user_reviews_summary_row')
        reviews = reviews.text.strip().replace('\r','').replace('\t','').replace('\n','') if reviews else None
        tags_div = soup1.find('div', class_='glance_tags popular_tags')
        tags = tags_div.find_all('a', class_='app_tag') if tags_div else 'N/A'

        if tags != 'N/A':
            tagged = [tag.text.strip() for tag in tags]
            tagged_string = ', '.join(tagged)
        else:
            tagged_string = 'N/A'

       
        data.append([description, reviews, tagged_string])
        
        counter+=1
    

100%|██████████| 10000/10000 [4:04:29<00:00,  1.47s/it]


In [5]:
data_df = pd.DataFrame(data, columns=['description','reviews','tags'])
data_df

,description,reviews,tags
0,About This GameThe Horologist's Legacy is a de...,All Reviews:7 user reviews- Need more user rev...,"Indie, Horror, Psychological Horror, Psycholog..."
1,About This GameMODERN COMBAT 5 IS AN INTENSE F...,Recent Reviews:Mixed(57)- 45% of the 57 user r...,"Free to Play, FPS, Action, Shooter, Massively ..."
2,About This GameIntelligence: Underwater Kingdo...,All Reviews:6 user reviews- Need more user rev...,"Puzzle, Underwater, Fishing, Card Game, Tablet..."
3,"About This GameDRAW, PLAY AND BLUFF, FOR ONLY ...",All Reviews:Very Positive(175)- 89% of the 175...,"Strategy, Card Game, Board Game, Multiplayer, ..."
4,About This GamePlay as a serial killer and use...,All Reviews:1 user reviews- Need more user rev...,"Action, Indie, Violent"
...,...,...,...
9995,"About This GameSTORY The year is 2021, America...",All Reviews:Very Positive(480)- 93% of the 480...,"Memes, Anime, Visual Novel, Nudity, Sexual Con..."
9996,About This Game****This game is on Alpha test ...,All Reviews:Mostly Negative(29)- 27% of the 29...,"RPG, Nudity, Adventure, Indie, Early Access, G..."
9997,"About This GameIn this VR exclusive game, trav...",All Reviews:2 user reviews- Need more user rev...,"Casual, VR"
9998,About This GameExperience real casino flair on...,All Reviews:Mixed(11)- 63% of the 11 user revi...,"Casual, Solitaire, Card Game, Puzzle, Strategy..."


In [6]:
data_df.to_csv('40to50_k_games.csv', index=False)